# Robustness Results Viewer (with Pareto & Vol-Carry)
Set OVERRIDE to your sweep folder (contains `summary.csv`).


In [ ]:
from pathlib import Path
import pandas as pd, plotly.express as px
OVERRIDE = None  # e.g. r'C:\Users\speed\Desktop\Forex CFD\runs\robustness_smoketest'
def find_latest():
    here = Path.cwd()
    cands = []
    for base in [here] + list(here.parents):
        r = base/'runs'
        if r.exists(): cands += list(r.glob('robustness_*'))
    return sorted(cands, key=lambda p: p.stat().st_mtime)[-1] if cands else None
root = Path(OVERRIDE) if OVERRIDE else find_latest()
if not root or not (root/'summary.csv').exists():
    display(pd.DataFrame({'hint':["No robustness results found.", "Run a sweep or set OVERRIDE."]}))
else:
    print('Using:', root)
    df = pd.read_csv(root/'summary.csv')
    hover_cols = [c for c in ['lookbacks','w_tsmom','w_xsec','w_mr','w_volcarry','loo'] if c in df.columns]
    display(df.sort_values('sharpe', ascending=False).head())
    fig = px.scatter(df, x='maxdd', y='sharpe', color='target_vol', hover_data=hover_cols)
    fig.update_layout(title='Sharpe vs MaxDD (color=target_vol)'); fig.show()
    # Pareto
    import numpy as np
    dx = df['maxdd'].abs(); dy = df['sharpe']
    order = np.lexsort((-dy.values, dx.values))
    mask = [False]*len(df); best_s=-1e9; best_d=1e9
    for i in order:
        d=float(dx.iloc[i]); s=float(dy.iloc[i])
        if d<=best_d and s>=best_s:
            mask[i]=True; best_d=d; best_s=s
    pf = df[mask]
    fig2 = px.scatter(pf, x='maxdd', y='sharpe', color='target_vol', hover_data=hover_cols)
    fig2.update_layout(title='Pareto Frontier (min |DD|, max Sharpe)'); fig2.show()
